## Deep Learning
a. Build a DNN with five hidden layers of 100 neurons each, He initialization, and the ELU activation function.

b. Using Adam optimization and early stopping, try training it on MNIST but only on digits 0 to 4, as we will use transfer learning for digits 5 to 9 in the next exercise. You will need a softmax output layer with five neurons, and as always make sure to save checkpoints at regular intervals and save the final model so you can reuse it later.

c. Tune the hyperparameters using cross-validation and see what precision you can achieve.

d. Now try adding Batch Normalization and compare the learning curves: is it converging faster than before? Does it produce a better model?

e. Is the model overfitting the training set? Try adding dropout to every layer and try again. Does it help?

In [1]:
import numpy as np
import tensorflow as tf

#### Build a DNN with five hidden layers of 100 neurons each, He initialization, and the ELU activation function.

In [2]:
# Create Inputs and outputs
n_inputs = 28 * 28
n_hidden = 100
n_outputs = 5

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")
is_training = tf.placeholder(tf.bool, shape=(), name='is_training')
bn_params = {
    'is_training': is_training,
    'decay': 0.99,
    'updates_collections': None
}
keep_prob = 0.8

In [3]:
# build layers
from tensorflow.contrib.layers import fully_connected
from tensorflow.contrib.layers import batch_norm
from tensorflow.contrib.layers import dropout

enable_dropout = False
he_init = tf.contrib.layers.variance_scaling_initializer()
with tf.contrib.framework.arg_scope(
        [fully_connected],
        weights_initializer = he_init,
        normalizer_fn =  batch_norm,
        normalizer_params = bn_params):
    if enable_dropout: 
        hidden_1 = fully_connected(X, n_hidden, scope='hidden1', activation_fn=tf.nn.relu)
        hidden_drop_1 = dropout(hidden_1, keep_prob, is_training=is_training)
        hidden_2 = fully_connected(hidden_drop_1, n_hidden, scope='hidden2', activation_fn=tf.nn.relu)
        hidden_drop_2 = dropout(hidden_2, keep_prob, is_training=is_training)
        hidden_3 = fully_connected(hidden_drop_2, n_hidden, scope='hidden3', activation_fn=tf.nn.relu)
        hidden_drop_3 = dropout(hidden_3, keep_prob, is_training=is_training)
        hidden_4 = fully_connected(hidden_drop_3, n_hidden, scope='hidden4', activation_fn=tf.nn.relu)
        hidden_drop_4 = dropout(hidden_4, keep_prob, is_training=is_training)
        logits = fully_connected(hidden_drop_4, n_outputs, scope='hidden5', activation_fn=tf.nn.softmax)
    else:
        hidden_1 = fully_connected(X, n_hidden, scope='hidden1', activation_fn=tf.nn.relu)
        hidden_2 = fully_connected(hidden_1, n_hidden, scope='hidden2', activation_fn=tf.nn.relu)
        hidden_3 = fully_connected(hidden_2, n_hidden, scope='hidden3', activation_fn=tf.nn.relu)
        hidden_4 = fully_connected(hidden_3, n_hidden, scope='hidden4', activation_fn=tf.nn.relu)
        logits = fully_connected(hidden_4, n_outputs, scope='hidden5', activation_fn=tf.nn.softmax)
    

In [4]:
# calculate loss
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')

In [5]:
# Optimizer
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer()
    train_op = optimizer.minimize(loss, name='train_op')

In [6]:
# evalue, 这一步不影响训练流程，只是为了获取一些信息
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [7]:
# init
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [8]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [14]:

n_epoch = 50
batch_size = 50
minimum_loss = 1.11
# minimum_loss = 0.913
# minimum_loss = 1.49

def run_train(sess, X_train, y_train, train_idx):
    sess.run(init)
#     sess.run(he_init)
    model_path = '/Users/chancezhang/machine_learning/hands_on_machine_learning/tmp/C11/C11_Ex08_' + str(train_idx) + '.ckpt'
    for epoch in range(n_epoch):
        for idx in range(X_train.shape[0] // batch_size):
            X_batch = X_train[batch_size * idx : batch_size * (idx + 1)]
            y_batch = y_train[batch_size * idx : batch_size * (idx + 1)]
            run_ret = sess.run(train_op, feed_dict={X:X_batch, y:y_batch, is_training:True})
        runtime_loss = loss.eval(feed_dict={X:X_train, y:y_train, is_training:False})
        runtime_acc = accuracy.eval(feed_dict={X:X_train, y:y_train, is_training:False})
        print('epoch ', epoch, 'runtime-accuracy:', runtime_acc, 'loss:', runtime_loss)
        if runtime_loss < minimum_loss:
            save_path = saver.save(sess, model_path) # early-stopping
            return;
    save_path = saver.save(sess, model_path) # early-stopping

# with tf.Session() as sess:
    
#     for epoch in range(n_epoch):
#         for mini_batch_idx in range(mnist.train.num_examples // batch_size):
#             X_batch, y_batch = mnist.train.next_batch(batch_size)
#             sess.run(train_op, feed_dict={X:X_batch, y:y_batch})
#         acc_train = accuracy.eval(feed_dict={X:X_batch, y:y_batch})
#         acc_test = accuracy.eval(feed_dict={X:mnist.test.images,
#                                             y:mnist.test.labels})
#         print('epoch ', epoch, 'train_acc:', acc_train, 'test_acc:', acc_test)
#     save_path = saver.save(sess, model_path)

In [15]:
# Cross Validation Implementation
split_size = 5
from sklearn.model_selection import KFold
results = []
with tf.Session() as sess:
    # 只采用0~4数据
    X_train_all = mnist.train.images
    y_train_all = mnist.train.labels
    selective_indices = np.where(y_train_all < 5)
    X_train = X_train_all[selective_indices]
    y_train = y_train_all[selective_indices]
    
    train_id = 0
    kf = KFold(n_splits=split_size)
    for train_idx, val_idx in kf.split(X_train, y_train):
        X_train_cs = X_train[train_idx]
        y_train_cs = y_train[train_idx]
        X_val_cs = X_train[val_idx]
        y_val_cs = y_train[val_idx]
        print('Begin Cross-Validation Train: ', train_id)
        run_train(sess, X_train_cs, y_train_cs, train_id)
        acc = accuracy.eval(feed_dict={X:X_val_cs, y:y_val_cs, is_training:False})
        print('accuracy: ', acc)
        results.append(acc)
        train_id += 1
print(results)

Begin Cross-Validation Train:  0
epoch  0 runtime-accuracy: 0.9844405 loss: 1.1319643
epoch  1 runtime-accuracy: 0.9910834 loss: 1.1255578
epoch  2 runtime-accuracy: 0.994115 loss: 1.120471
epoch  3 runtime-accuracy: 0.9954525 loss: 1.1166725
epoch  4 runtime-accuracy: 0.9967008 loss: 1.1102158
epoch  5 runtime-accuracy: 0.99772626 loss: 1.108918
accuracy:  0.98555636
Begin Cross-Validation Train:  1
epoch  0 runtime-accuracy: 0.98082924 loss: 1.1281295
epoch  1 runtime-accuracy: 0.9892555 loss: 1.1218494
epoch  2 runtime-accuracy: 0.9930896 loss: 1.121137
epoch  3 runtime-accuracy: 0.99465 loss: 1.1174667
epoch  4 runtime-accuracy: 0.99621046 loss: 1.1157897
epoch  5 runtime-accuracy: 0.9943825 loss: 1.1176897
epoch  6 runtime-accuracy: 0.99621046 loss: 1.1093544
accuracy:  0.989301
Begin Cross-Validation Train:  2
epoch  0 runtime-accuracy: 0.98319215 loss: 1.1223212
epoch  1 runtime-accuracy: 0.99028087 loss: 1.1171403
epoch  2 runtime-accuracy: 0.9934017 loss: 1.1157624
epoch  3 ru

In [16]:
# Train and save final model
with tf.Session() as sess:
    print('Begin Final Train')
    X_train_all = mnist.train.images
    y_train_all = mnist.train.labels
    selective_indices = np.where(y_train_all < 5)
    X_train = X_train_all[selective_indices]
    y_train = y_train_all[selective_indices]
    
    X_test_all = mnist.test.images
    y_test_all = mnist.test.labels
    selective_indices = np.where(y_test_all < 5)
    X_test = X_test_all[selective_indices]
    y_test = y_test_all[selective_indices]
    
    run_train(sess, X_train, y_train, 100)
    acc = accuracy.eval(feed_dict={X:X_test, y:y_test, is_training:False})
    print('Final Accuracy: ', acc)

Begin Final Train
epoch  0 runtime-accuracy: 0.9844853 loss: 1.128446
epoch  1 runtime-accuracy: 0.9910122 loss: 1.125555
epoch  2 runtime-accuracy: 0.9937585 loss: 1.1201246
epoch  3 runtime-accuracy: 0.99493545 loss: 1.1182389
epoch  4 runtime-accuracy: 0.9961481 loss: 1.1138163
epoch  5 runtime-accuracy: 0.99604106 loss: 1.1080765
Final Accuracy:  0.99085426
